In [ ]:
!nvidia-smi

Tue Dec 15 15:29:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---
Modified Fasterrcnn

In [ ]:
!mkdir drive/MyDrive/AIIntro/[processed]dream_picture/20200610-20200620-camera-3/tuning_photos/Sharpen_0p4_Add_5/Maskrcnn_test_thresh-0p35/

In [ ]:
import gdown
url = 'https://drive.google.com/uc?export=download&id=170CjxBy-5nkoCmVzkoSfOXcGxZ-ZqwET'
output = 'drive/MyDrive/AIIntro/[processed]dream_picture/20200610-20200620-camera-3/tuning_photos/Sharpen_0p4_Add_5/Maskrcnn_test_thresh-0p35/result.json'
gdown.download(url, output, quiet=False) 

Downloading...
From: https://drive.google.com/uc?export=download&id=170CjxBy-5nkoCmVzkoSfOXcGxZ-ZqwET
To: /content/drive/MyDrive/AIIntro/[processed]dream_picture/20200610-20200620-camera-3/tuning_photos/Sharpen_0p4_Add_5/Maskrcnn_test_thresh-0p35/result.json
0.00B [00:00, ?B/s]


'drive/MyDrive/AIIntro/[processed]dream_picture/20200610-20200620-camera-3/tuning_photos/Sharpen_0p4_Add_5/Maskrcnn_test_thresh-0p35/result.json'

In [ ]:
# MODIFY THE THRESHOLD OF FASTERRCNN TO 0.4
# DO DETECTION WITH Sharp_0p4_Add_5

In [ ]:
# Copyright (c) 2017-present, Facebook, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Function `vis_bbox`, `vis_mask`, and `vis_class` are adapted from: 
# https://github.com/facebookresearch/Detectron/blob/7aa91aaa5a85598399dc8d8413e05a06ca366ba7/detectron/utils/vis.py

"""PyTorch object detection example."""

import imageio
import os
import json

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as trns
from PIL import Image

_GRAY = (218, 227, 218)
_GREEN = (18, 127, 15)
_WHITE = (255, 255, 255)

COCO_INSTANCE_CATEGORY_NAMES = [
    "__background__", "person", "bicycle", "car", "motorcycle", "airplane", "bus",
    "train", "truck", "boat", "traffic light", "fire hydrant", "N/A", "stop sign",
    "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow",
    "elephant", "bear", "zebra", "giraffe", "N/A", "backpack", "umbrella", "N/A", "N/A",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "N/A", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
    "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza",
    "donut", "cake", "chair", "couch", "potted plant", "bed", "N/A", "dining table",
    "N/A", "N/A", "toilet", "N/A", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "N/A", "book",
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]


def vis_bbox(image, bbox, color=_GREEN, thick=1):
    """Visualizes a bounding box."""
    image = image.astype(np.uint8)
    bbox = list(map(int, bbox))
    x0, y0, x1, y1 = bbox
    cv2.rectangle(image, (x0, y0), (x1, y1), color, thickness=thick)
    return image


def vis_mask(image, mask, col, alpha=0.4, show_border=True, border_thick=1):
    """Visualizes a single binary mask."""
    image = image.astype(np.float32)

    mask = mask >= 0.5
    mask = mask.astype(np.uint8)
    idx = np.nonzero(mask)

    image[idx[0], idx[1], :] *= 1.0 - alpha
    image[idx[0], idx[1], :] += alpha * col

    if show_border:
        contours = cv2.findContours(
            mask.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)[-2]
        cv2.drawContours(image, contours, -1, _WHITE,
                         border_thick, cv2.LINE_AA)

    return image.astype(np.uint8)


def vis_class(image, bbox, text, bg_color=_GREEN, text_color=_GRAY, font_scale=0.35):
    """Visualizes the class."""
    image = image.astype(np.uint8)
    x0, y0 = int(bbox[0]), int(bbox[1])

    # Compute text size
    font = cv2.FONT_HERSHEY_SIMPLEX
    ((text_w, text_h), _) = cv2.getTextSize(text, font, font_scale, 1)

    # Place text background
    back_tl = x0, y0 - int(1.3 * text_h)
    back_br = x0 + text_w, y0
    cv2.rectangle(image, back_tl, back_br, bg_color, -1)

    # Show text
    text_tl = x0, y0 - int(0.3 * text_h)
    cv2.putText(image, text, text_tl, font, font_scale,
                text_color, lineType=cv2.LINE_AA)

    return image


def run_object_detection(model, image_path, transforms, output_path, threshold):
    """Inference."""
    ### modified by sky712345678, add support for run object detection in batch
    ### modified by sky712345678, add support for generate detection results in JSON format
    ### modified by sky712345678, save detection result with imageio to make this easier to use in Google Colab
    # os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    imageAddress = []
    for f in os.listdir(image_path):
        if(f.__contains__('.jpg')):
            imageAddress.append(image_path + '/' + f)
    
    imageAddress.sort()

    json_result = []
    frame_id = 1

    for f in imageAddress:
        # Read image and run prepro
        image = Image.open(f).convert("RGB")
        image_tensor = transforms(image)
        # print(f"\n\nImage size after transformation: {image_tensor.size()}")

        image_tensor = image_tensor.to(device)

        # Feed input and get results at index 0
        # (input image is at index 0 in the list)
        outputs = model([image_tensor])[0]

        # Result postpro and vis
        display_image = np.array(image)
        outputs = {k: v.cpu().numpy() for k, v in outputs.items()}
        is_mask = True if "masks" in outputs else False
        if is_mask:
            outputs["masks"] = np.squeeze(outputs["masks"], axis=1)

        # print("\n\nInference results:")
        json_objects = []
        for i, (bbox, label, score) in enumerate(zip(outputs["boxes"], outputs["labels"], outputs["scores"])):
            if score < threshold:
                continue

            # print(f"Label {label}: {COCO_INSTANCE_CATEGORY_NAMES[label]} ({score:.2f})")

            object_dict = {
                "class_id": int(label),
                "name": COCO_INSTANCE_CATEGORY_NAMES[label],
                "confidence": float(score)
            }
            json_objects.append(object_dict)

            display_image = vis_bbox(display_image, bbox)
            display_image = vis_class(
                display_image, bbox, COCO_INSTANCE_CATEGORY_NAMES[label])
            if is_mask:
                display_image = vis_mask(
                    display_image, outputs["masks"][i], np.array([0., 0., 255.]))

        '''plt.figure(figsize=(10, 6))
        plt.imshow(display_image)
        plt.xticks([])
        plt.yticks([])
        plt.savefig(output_path + '/' + f, bbox_inches="tight")'''
        imageio.imsave(f.replace(image_path, output_path), display_image)

        result_dict = {
            "frame_id": frame_id,
            "filename": f,
            "objects": json_objects
        }
        json_result.append(result_dict)
        frame_id += 1

    json_resultDump = json.dumps(json_result, indent=4)
    with open(output_path + '/result.json', 'w') as outfile:
        outfile.write(json_resultDump)
    print('\n\ncheck result file saved successfully!')
    ### modified by sky712345678, add support for run object detection in batch
    ### modified by sky712345678, add support for generate detection results in JSON format
    ### modified by sky712345678, save detection result with imageio to make this easier to use in Google Colab

    return


if __name__ == "__main__":
    ### modified by sky712345678, comment out argparse because it isn't useful in Google Colab
    # parser = argparse.ArgumentParser("PyTorch Object Detection")
    # parser.add_argument("--image_path", type=str,
    #                     default="images/sheep-herd-shepherd-hats-dog-meadow.jpg", help="path to image")
    # parser.add_argument("--model_type", type=str,
    #                     default="fasterrcnn", help="fasterrcnn or maskrcnn")
    # parser.add_argument("--output_path", type=str,
    #                     default="out.png", help="path to save output image")

    # Parse arguments
    # args = parser.parse_args()

    image_path = 'drive/MyDrive/AIIntro/[processed]dream_picture/20200610-20200620-camera-3/tuning_photos/Sharpen_0p4_Add_5'
    model_type = 'maskrcnn'
    output_path = 'drive/MyDrive/AIIntro/[processed]dream_picture/20200610-20200620-camera-3/tuning_photos/Sharpen_0p4_Add_5/Maskrcnn_test_thresh-0p35'
    threshold = 0.35
    ### modified by sky712345678, comment out argparse because it isn't useful in Google Colab

    # Define image transforms
    transforms = trns.ToTensor()

    # Load model
    if model_type == "fasterrcnn":
        net = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    elif model_type == "maskrcnn":
        net = models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    else:
        raise AssertionError

    ### modified by sky12345678, add GPU acceleration support
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = net.to(device)
    ### modified by sky12345678, add GPU acceleration support

    # print(model)

    # Set model to eval mode
    model.eval()


    # Run model
    with torch.no_grad():
        run_object_detection(model, image_path, transforms, output_path, threshold)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth





check result file saved successfully!


---
**jwyang faster rcnn (unusable)**

In [ ]:
!git clone -b pytorch-1.0 https://github.com/jwyang/faster-rcnn.pytorch.git

In [ ]:
cd faster-rcnn.pytorch

In [ ]:
!mkdir data

In [ ]:
!pip install -r requirements.txt

In [ ]:
!wget https://www.dropbox.com/s/iev3tkbz5wyyuz9/resnet101_caffe.pth?dl=0

In [ ]:
!wget https://www.dropbox.com/s/s3brpk0bdq60nyb/vgg16_caffe.pth?dl=0

In [ ]:
!mv vgg16_caffe.pth?dl=0 vgg16.pth

In [ ]:
!mkdir data/pretrained_model

In [ ]:
!mv vgg16.pth data/pretrained_model/

In [ ]:
cd lib

In [ ]:
!python setup.py build develop

In [ ]:
cd ..

In [ ]:
!pip install scipy==1.2.1

In [ ]:
!python demo.py --net vgg16 \
               --checksession 1 --checkepoch 6 --checkpoint 416 \
               --cuda --load_dir data/pretrained_mode